<a href="https://colab.research.google.com/github/muhammadolammi/llms-from-scratch/blob/main/gpt_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from datasets import load_dataset
import pickle
import mmap
import random
import os


In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
BLOCK_SIZE =64
BATCH_SIZE = 128
MAX_ITER = 500
LEARNING_RATE = 3e-4
EVAL_ITERS = 50
N_EMBD = 384
N_LAYERS = 8
N_HEAD = 8
DROP_OUT =0.2
MODEL_PKL_DIR ="drive/MyDrive/GPT/models/model01.pkl"
DATA_fOLDER="/content/drive/MyDrive/Colab Datasets/openwebtext/openwebtext"

# book_dir = "drive/MyDrive/Colab Datasets/wizard of oz.txt"
print(device)


cuda


In [3]:
chars =""
with open(f"{DATA_fOLDER}/vocab.txt", "r" ) as f:
  text = f.read()
  chars = sorted(list(set(text)))

In [4]:
chars = sorted(set(chars + ['[UNK]']))

vocab_size = len(chars)
vocab_size

5524

In [5]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int.get(c, string_to_int['[UNK]']) for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])


In [6]:
# memory map for using small snippets of text from a single file of any size
def get_random_chunk(split, ):

    filename = os.path.join(DATA_fOLDER, "train_split.txt") if split == 'train' else os.path.join(DATA_fOLDER, "val_split.txt")
    with open(filename, 'rb') as f:
        with mmap.mmap(f.fileno(), 0, access=mmap.ACCESS_READ) as mm:
            # Determine the file size and a random position to start reading
            file_size = len(mm)
            start_pos = random.randint(0, (file_size) - BLOCK_SIZE*BATCH_SIZE)

            # Seek to the random position and read the block of text
            mm.seek(start_pos)
            block = mm.read(BLOCK_SIZE*BATCH_SIZE-1)

            # Decode the block to a string, ignoring any invalid byte sequences
            decoded_block = block.decode('utf-8', errors='ignore').replace('\r', '')

            # Train and test splits
            data = torch.tensor(encode(decoded_block), dtype=torch.long)

    return data


def get_batch(split):
    split = split.lower()
    data = get_random_chunk(split)
    ix = torch.randint(len(data)- BLOCK_SIZE, (BATCH_SIZE,))
    # print(ix)
    # to get our batch we just stack bocks on each other
    x = torch.stack([data[i:i+BLOCK_SIZE] for i in ix] )
    y = torch.stack([data[i+1:i+BLOCK_SIZE+1] for i in ix] )
    x,y = x.to(device), y.to(device)
    return x,y


In [7]:
class Head(nn.Module):
  def __init__(self,head_size):
     super().__init__()
     # create k, q, v using linear transformation
     self.key = nn.Linear(N_EMBD, head_size, bias=False)
     self.query = nn.Linear(N_EMBD, head_size, bias=False)
     self.value = nn.Linear(N_EMBD, head_size, bias=False)
     self.register_buffer('tril', torch.tril(torch.ones(BLOCK_SIZE, BLOCK_SIZE)))

     self.dropout = nn.Dropout(DROP_OUT)


  def forward(self, x):


    # input  size (batch, time-step, channels)
    # output  size (batch, time-step, head size)
    B,T,C = x.shape

    # get k and q
    k = self.key(x) # (B,T,hs)
    q = self.query(x) # (B,T,hs)

    # calculating attention scores using dot scaled product
    #we transpose k and exchange index 1,2
    wei = q @ k.transpose(-2,-1)  # (B, T, hs) @ (B, hs, T) -> (B, T, T)
    # scale by 1/sqrt(len of k/q rows)
    wei =  wei /  k.shape[-1] ** 0.5
    wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
    wei = F.softmax(wei, dim=-1) # (B, T, T)
    wei = self.dropout(wei)
    # perform the weighted aggregation of the values
    v = self.value(x) # (B,T,hs)
    out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
    return out # (B, T, hs)



class MultiHeadAttention(nn.Module):
  def __init__(self, num_heads ):
     super().__init__()
     self.heads = nn.ModuleList( [ Head(num_heads) for _ in range(N_HEAD)])
     # projection is the linear transformation in the multihead
     self.proj = nn.Linear(num_heads*N_HEAD, N_EMBD)
     ## add a dropout
     self.dropout = nn.Dropout(DROP_OUT)
  def forward(self, x):
    ## concat n-head dotproduct attention results by features i.e concat them in the feature dimension
    out = torch.concat([head(x) for head in self.heads], dim=-1)
    ## add linear
    out = self.proj(out)
    ## dropout
    out = self.dropout(out)
    return out


class FeedForward(nn.Module):
  def __init__(self, ):
     super().__init__()
     self.net = nn.Sequential(
         nn.Linear(N_EMBD, 4* N_EMBD),
         nn.ReLU(),
         nn.Linear(4*N_EMBD,  N_EMBD),
         nn.Dropout(DROP_OUT)


     )
  def forward(self, x):
    return self.net(x)



class LAYER(nn.Module):
  def __init__(self):
     super().__init__()
     self.ff = FeedForward()
     head_size = N_EMBD // N_HEAD
     self.sa = MultiHeadAttention(head_size)
     self.ln1 = nn.LayerNorm(N_EMBD)
     self.ln2 = nn.LayerNorm(N_EMBD)
  def forward(self, x):
    ## x1 is the output from self attention
    x1 = self.sa(x)
    # we do a residual connection here, by adding initial x and  the result from sa then normalizing
    x = self.ln1(x1+x)
    #x1 is the output from feed forwad
    x1 = self.ff(x)
    # we do a residual connection here, by adding initial x and  the result from feed forward then normalizing
    x = self.ln1(x1+x)
    return x

class GPTLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, N_EMBD)
        self.position_embedding_table = nn.Embedding(BLOCK_SIZE, N_EMBD)
        ## create the sequential decoder layers
        self.layers = nn.Sequential(*[LAYER() for _ in range(N_LAYERS)])

        # final layer norm function for add & norm
        self.ln_f = nn.LayerNorm(N_EMBD)

        # linear head
        self.lm_head = nn.Linear(N_EMBD, vocab_size)

        # initialize weights
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, index, targets=None):
        # forward function, should return logits and the loss of current layer/run
        # instead of getting logit randomly using embedding(As we did with Bigram)
        # now we can use our transformers
        # logits = self.token_enbedding_table(index)
        # B = BATCH SIZE
        # T = BLOCK SIZE
        #C=N_EMBD
        B, T = index.shape
        # embed input
        tok_embd = self.token_embedding_table(index) # (B,T,C)
        # get pos embeding/encoding for
        positions = torch.arange(T, device=device)
        position_embd = self.position_embedding_table(positions) # (T, C)
        # add pos embd and tokenembd
        # we can add since # (B,T,C)   + (T, C) is the broadcast rule cause (T, C) is just treated as (1, T, C)
        x = tok_embd +  position_embd # (B,T,C)
        # send x to decode layers
        x = self.layers(x) # (B,T,C)
        # add the layer norm
        x = self.ln_f(x) # (B,T,C)
        #get logits with linear  head
        logits = self.lm_head(x) # (B,T,C)

        if targets==None:
          ## if no target, just return logit and None for loss
          return logits, None
        else:

          # WE need to reshape logit to use the cross_entropy loss function

          B, T, C = logits.shape
          # loss = F.cross_entropy(logit.view(B*T, C), targets.view(B*T))
          #we can actually run this ^^ fine and logit will still be 3 dimensional, according to the tutorila
          # when theres a target logit should return 2 dimension , so will run these VV for now
          logits = logits.view(B*T, C)
          loss = F.cross_entropy(logits, targets.view(B*T))
          return logits, loss

    # generate generate tokens.
    def generate(self, index, max_new_tokens):
      # index is (B,T) array of index in current context
      for _ in range(max_new_tokens):
        # crop idx to the last block_size tokens
        index_cond = index[:, -BLOCK_SIZE:]
        # generate predictions

        logits, loss = self.forward(index_cond)
        # target is none here, so logits is in (B, T, C)
        # get logits for last T

        logits = logits[:,-1,:] # becomes (B, C)

        # get probabilities for these logits/predictions using softmax
        probs = F.softmax(logits, dim=-1) # return (B,C)
        # sample from the probabilities distrubution
        index_next = torch.multinomial(probs, num_samples=1)

        # add sampled index to running sequence
        index = torch.cat((index, index_next), dim=1) # becomes (B , T+1)
      # return the final index
      return index









In [8]:
model = GPTLanguageModel()
print("loading model parameters")
with open(MODEL_PKL_DIR, "rb") as f:
  model = pickle.load(f)
print("loaded succesfully.")

m = model.to(device)

loading model parameters
loaded succesfully.


In [9]:
## loss report

@torch.no_grad()

def estimate_loss():
  out ={}
  model.eval()
  for split in ["train", "val"]:
    losses = torch.zeros(EVAL_ITERS)
    for k in range(EVAL_ITERS):
      X, Y = get_batch(split)
      logits, loss  = model(X,Y)
      losses[k] = loss.item()
    out[split] = losses.mean()
  model.train()
  return out





In [10]:
## optimizer

optimizer = torch.optim.AdamW(model.parameters(),lr=LEARNING_RATE )

for iter in range(MAX_ITER):
  if iter % EVAL_ITERS ==0:
    # evaluate here
    losses = estimate_loss()
    print(f"steps: {iter}, train loss: {losses['train']:.4f}, val loss: {losses['val']:.4f}")

  # optimize


  # get batch of train data
  xb, yb = get_batch("train")

  # evaluate the loss
  logits, loss = model.forward(xb,yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()
print(loss.item())


with open(MODEL_PKL_DIR, "wb") as f:
  pickle.dump(model,f)


steps: 0, train loss: 1.5028, val loss: 1.5421
steps: 50, train loss: 1.5152, val loss: 1.5954
steps: 100, train loss: 1.5871, val loss: 1.5194
steps: 150, train loss: 1.5280, val loss: 1.4794
steps: 200, train loss: 1.5384, val loss: 1.5552
steps: 250, train loss: 1.4847, val loss: 1.5017
steps: 300, train loss: 1.4677, val loss: 1.5049
steps: 350, train loss: 1.4766, val loss: 1.5025
steps: 400, train loss: 1.5088, val loss: 1.5332
steps: 450, train loss: 1.5212, val loss: 1.5355
1.3842111825942993


In [14]:
prompt = 'Hello! Can you see me?'
context = torch.tensor(encode(prompt), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context.unsqueeze(0), max_new_tokens=100)[0].tolist())
print(generated_chars)

Hello! Can you see me? b." Barie sqatroo rizeemmonistriboutside careinranty Fluender 20000 and BE caling Helwanlay, Selour
